# Load Dataset

In [1]:
import pandas as pd

train_df = pd.read_csv("../../datasets/feature_engineering/train.csv", index_col="PassengerId")
test_df = pd.read_csv("../../datasets/feature_engineering/test.csv", index_col="PassengerId")

# Split Dataset (Train and Test)

In [2]:
from sklearn.model_selection import train_test_split

X_train = train_df.drop(columns=["age", "pclass", "name", "ticket", "cabin", "deck", "no_ticket", "is_alone", "survived"])
y_train = train_df.survived

X_test = test_df.drop(columns=["age", "pclass", "name", "ticket", "cabin", "deck", "is_alone", "no_ticket"])

print(f"X_train shape : {X_train.shape}")
print(f"X_train shape : {y_train.shape}")
print(f"X_test shape  : {X_test.shape}")

X_train shape : (891, 10)
X_train shape : (891,)
X_test shape  : (418, 10)


## Preprocessing Data Pipeline

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer, PowerTransformer, RobustScaler
from sklearn.compose import ColumnTransformer, make_column_selector

numerical_prep_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", RobustScaler(quantile_range=(5., 80.)))
    ],
    verbose=1
)

categorical_prep_pipeline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder())
    ],
    verbose=1
)

preprocess_pipeline = ColumnTransformer(
    transformers=[
        ("step1_numerical_pipeline", numerical_prep_pipeline, make_column_selector(dtype_include="number")),
        ("step2_categorical_pipeline", categorical_prep_pipeline, make_column_selector(dtype_include="object"))
    ],
    verbose=1, verbose_feature_names_out=True
)

## Model Pipeline

In [4]:
from xgboost import XGBClassifier

model_pipeline = Pipeline(
    steps=[
        ("step1_preprocess_pipeline", preprocess_pipeline),
        ("step2_algo", XGBClassifier(n_jobs=-1))
    ],
    verbose=1
)

# Train

In [5]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    "step2_algo__n_estimators": [100, 200, 300, 400, 500],
    "step2_algo__learning_rate": [0.125, 0.25, 0.5, 1., 2.],
    "step2_algo__booster": ["gbtree", "gblinear", "dart"],
    "step2_algo__tree_method": ["exact", "approx", "hist"],
    "step2_algo__gamma": [0.125, 0.25, 0.5, 1., 2.]    
}

model = RandomizedSearchCV(estimator=model_pipeline, param_distributions=params, cv=3, scoring="accuracy", n_jobs=-1, verbose=1)
model.fit(X_train, y_train);

pd.DataFrame(model.cv_results_).sort_values(by="rank_test_score").iloc[:5, :]

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing scaler, total=   0.0s
[ColumnTransformer]  (1 of 2) Processing step1_numerical_pipeline, total=   0.0s
[Pipeline] ........... (step 1 of 2) Processing imputer, total=   0.0s
[Pipeline] ........... (step 2 of 2) Processing encoder, total=   0.0s
[ColumnTransformer]  (2 of 2) Processing step2_categorical_pipeline, total=   0.0s
[Pipeline]  (step 1 of 2) Processing step1_preprocess_pipeline, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing step2_algo, total= 2.4min


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_step2_algo__tree_method,param_step2_algo__n_estimators,param_step2_algo__learning_rate,param_step2_algo__gamma,param_step2_algo__booster,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,5.933635,0.535526,0.021161,0.002154,approx,300,0.125,2.0,gbtree,"{'step2_algo__tree_method': 'approx', 'step2_a...",0.808081,0.858586,0.838384,0.835017,0.020756,1
7,8.050634,0.428029,0.022518,0.002200,approx,500,0.25,2.0,gbtree,"{'step2_algo__tree_method': 'approx', 'step2_a...",0.801347,0.851852,0.841751,0.831650,0.021821,2
3,25.289486,1.171854,0.537855,0.038842,hist,300,0.25,0.25,dart,"{'step2_algo__tree_method': 'hist', 'step2_alg...",0.804714,0.831650,0.838384,0.824916,0.014547,3
0,4.393888,0.729186,0.024670,0.000669,approx,100,0.125,0.25,gblinear,"{'step2_algo__tree_method': 'approx', 'step2_a...",0.811448,0.821549,0.838384,0.823793,0.011111,4
1,3.399864,0.294187,0.024437,0.002461,hist,500,0.125,0.25,gblinear,"{'step2_algo__tree_method': 'hist', 'step2_alg...",0.811448,0.821549,0.838384,0.823793,0.011111,4


# Save Model and Prediction

## Model

In [6]:
from datetime import datetime
from joblib import dump, load

now = datetime.now()
now = now.strftime("%m_%d_%Y-%H_%M_%S")
model_name = str(model.estimator.named_steps.step2_algo).split("(")[0]

dump(value=model, filename="../../pretrained_models/" + now + "_" + model_name +".joblib")
model = load(filename="../../pretrained_models/" + now + "_" + model_name + ".joblib")

## Prediction

In [7]:
y_test_preds = model.predict(X=X_test)

pred_df = pd.DataFrame({
    "PassengerId": X_test.index,
    "Survived": y_test_preds
})

pred_df.to_csv("../../submissions/" + now + "_" + model_name + ".csv", index=False)